In [2]:
from tensorflow import keras as tfk
import tensorflow as tf

In [3]:
# SFace is Robust, becauae it uses a different loss function - Sigmoid Constrained Hypersphere Loss (SCH)

def sigmoid_constrained_hypersphere_loss(y_true, y_pred, alpha=1.0, m=1.0):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # Compute the cosine similarity between the feature vectors and the weight vectors
    cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)

    # Compute the predicted probabilities
    p = (1.0 + alpha * y_true * cosine_sim) / (1.0 + alpha)

    # Compute the loss
    loss = -tf.reduce_mean(tf.math.log(p + 1e-12))

    return loss


The SFace model uses the above loss function, on the MobileFacenet architecture ~

In [4]:
# MobileFaceNet Architecture ~
def conv_block(input, n_filters, kernel_size, stride=1, padding='same'):
    x = tfk.layers.Conv2D(n_filters, kernel_size, stride, padding)(input)
    x = tfk.layers.BatchNormalization(epsilon=0.001)(x)
    x = tfk.layers.PReLU()(x)
    return x

def depthwise_conv_block(input, n_filters, kernel_size, stride=1, padding = 'same'):
    x = tfk.layers.SeparableConv2D(n_filters, kernel_size, stride, padding)(input)
    x = tfk.layers.BatchNormalization(epsilon=0.001)(x)
    x = tfk.layers.PReLU()(x)
    return x

def inverted_residual_block(input, n_filters, stride, expf):
    channel_axis = 1 if tfk.backend.image_data_format() == 'channels_first' else -1
    n_input_channels = tfk.backend.int_shape(input)[channel_axis]
    x = input
    #Expansion
    x = tfk.layers.Conv2D(n_input_channels * expf, kernel_size=1, padding='same')(x)
    x = tfk.layers.BatchNormalization(epsilon=0.001)(x)
    x = tfk.layers.PReLU()(x)

    #Depthwise
    x = tfk.layers.DepthwiseConv2D(kernel_size=3, strides=stride, padding='same')(x)
    x = tfk.layers.BatchNormalization(epsilon=0.001)(x)
    x = tfk.layers.PReLU()(x)

    #Projection
    x = tfk.layers.Conv2D(n_filters, kernel_size=1, padding='same')(x)
    x = tfk.layers.BatchNormalization(epsilon=0.001)(x)

    #Skip connection
    if n_input_channels == n_filters and stride == 1:
        x = tfk.layers.Add()([input, x])
    
    return x

def bottleneck(input, n_filters, stride, expf, repeat=1):
    for i in range(repeat):
        input = inverted_residual_block(input, n_filters, stride, expf)
    return input

# MobileFaceNet Model ~
def My_mobilefacenet(input_shape=(112,112,3), n_classes=512):
    input = tfk.layers.Input(shape=input_shape)
    x = conv_block(input, n_filters=64, kernel_size=3, stride=2)

    x = depthwise_conv_block(x, n_filters=64, kernel_size=3, stride=1)

    x = bottleneck(x, n_filters=64, stride=2, expf=2, repeat=5)
    x = bottleneck(x, n_filters=128, stride=2, expf=4, repeat=1)
    x = bottleneck(x, n_filters=128, stride=1, expf=2, repeat=6)
    x = bottleneck(x, n_filters=128, stride=2, expf=4, repeat=1)
    x = bottleneck(x, n_filters=128, stride=1, expf=2, repeat=2)
    
    x = conv_block(x, n_filters=512, kernel_size=1, stride=1)

    x = tfk.layers.GlobalAveragePooling2D()(x)

    x = tf.keras.layers.Dense(128)(x)

    outputs = tfk.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)

    model = tfk.models.Model(inputs=input, outputs=outputs)

    return model

model = My_mobilefacenet()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 56, 56, 64)   1792        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 56, 56, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [ ]:
# Get & Load datasets ~
train_data, test_data = [], []

In [10]:
model.compile(optimizer=tfk.optimizers.Adam(learning_rate=0.001),
                loss = sigmoid_constrained_hypersphere_loss,
                metrics=['accuracy'])


In [ ]:
history = model.fit(train_data,
                        epochs = 10,
                        steps_per_epoch = len(train_data),
                        validation_data = test_data,
                        validation_steps = len(test_data),
                        verbose = 1)